In [15]:
import librosa
from IPython.display import Audio
import pathlib
from pathlib import Path
from fastai.vision.all import *
from fastai.vision.widgets import *

In [16]:
path = Path('../')
model_path = path/'models'

try:
    learn_inf = load_learner(model_path/'sound_model_V1')
except:
    posix_backup = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
    learn_inf = load_learner(model_path/'sound_model_V1')
    pathlib.PosixPath = posix_backup
    
btn_upload = widgets.FileUpload(accept=".wav,.mp3")
out_pl = widgets.Output()
lbl_pred = widgets.Label()
lbl_pred.value = 'Please select audio'

In [17]:
def log_mel_spec_tfm(audio_upload, dst_path = path/'../data/imgs/uploaded'):
    data, sample_rate = librosa.load(io.BytesIO(audio_upload.data[-1]))
    
    n_fft = 1024
    hop_length = 512
    n_mels = 80
    fmin = 20
    fmax = sample_rate / 2 
    
    mel_spec_power = librosa.feature.melspectrogram(data, sr=sample_rate, n_fft=n_fft, 
                                                    hop_length=hop_length, 
                                                    n_mels=n_mels, power=2.0, 
                                                    fmin=fmin, fmax=fmax)
    
    #mel_spec_power = librosa.feature.melspectrogram(x, sr=sample_rate)
    
    mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max)
    
    
    dst_path.mkdir(exist_ok=True)
    fname = list(audio_upload.value)[0]
    
    plt.imsave(dst_path / (fname[:-4] + '.png'), mel_spec_db)
    
    return dst_path / (fname[:-4] + '.png')

In [18]:
def on_click_classify(change):
    out_pl.clear_output()
    with out_pl: display(Audio(btn_upload.data[-1]))
    dst_path = path/'../data/imgs/uploaded'
    audio_img = log_mel_spec_tfm(btn_upload, dst_path)
    pred,pred_idx,probs = learn_inf.predict(audio_img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [19]:
btn_upload.observe(on_click_classify, names=['data'])

In [20]:
widgets.VBox([widgets.Label('Select your audio'),
      btn_upload, out_pl, lbl_pred])

<ipython-input-17-b4d504f916d7>:10: FutureWarning: Pass y=[-5.1559568e-06 -1.4841992e-06  9.4970856e-06 ... -9.5554744e-04
 -8.0968329e-04 -6.3658389e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel_spec_power = librosa.feature.melspectrogram(data, sr=sample_rate, n_fft=n_fft,


In [13]:
#!pip install voila

In [14]:
#!jupyter serverextension enable voila --sys-prefix